In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" #! specify gpu here
import numpy as np
from rich import print
import torch
import cv2
import matplotlib.pyplot as plt
from torch import optim, nn
import torchvision
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import pytorch_lightning as pl
from torchmetrics.classification import BinaryAccuracy
import clip
import exp_utils as exp_utils
import albumentations as A
from data_loader import DataLoader
from data_loader_even_pairwise import DataLoaderEvenPairwise
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from IPython.display import display
from main import _transform

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# TODO: what does the CLIP encoder output?

seen_classes = ["hca_0", "hca_1", "hca_2", "hca_2a", "hca_3", "hca_4", "hca_5", "hca_6"]
unseen_classes = ["hca_7", "hca_8", "hca_9", "hca_10", "hca_11", "hca_11a", "hca_12"]

img_path = "datasets/2023-02-20_hca_backs"
preprocessing_path = "datasets/2023-02-20_hca_backs_preprocessing_opencv"

                        
# dl = DataLoader(img_path,
#                         preprocessing_path=preprocessing_path,
#                         batch_size=1,
#                         seen_classes=seen_classes,
#                         unseen_classes=unseen_classes,
#                         train_transform=train_transform,
#                         val_transform=val_transform,
#                         cuda=True)

val_transform = A.Compose([ToTensorV2()])
train_transform = A.Compose([ToTensorV2()])

dl = DataLoaderEvenPairwise(img_path,
                            preprocessing_path=preprocessing_path,
                            batch_size=1,
                            num_workers=8,
                            shuffle=True, #! usually True
                            seen_classes=seen_classes,
                            unseen_classes=unseen_classes,
                            train_transform=train_transform,
                            val_transform=val_transform)

====================

Imgs per class limited to 30

====================

====================

Imgs per class limited to 30

====================

====================

Imgs per class limited to 30

====================

In [3]:
from object_reid_superglue import ObjectReIdSuperGlue

#! WIP
model = "/home/sruiz/projects/reconcycle/superglue_training/output/train/2023-11-18_superglue_model/weights/best.pt"
# model = "indoor"
object_reid_superglue = ObjectReIdSuperGlue(model)




curr_weights_path /home/sruiz/projects/reconcycle/superglue_training/output/train/2023-11-18_
superglue_model/weights/best.pt

Loaded SuperPoint model
Loaded SuperGlue model ("/home/sruiz/projects/reconcycle/superglue_training/output/train/2023-11-18_superglue_model/weights/best.pt" weights)


In [4]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

visualise = False
debug = False

min_matches = 10

results = []
batch_prediction = []
for i, batch in enumerate(dl.dataloaders["seen_train"]):
    sample1, label1, dets1, sample2, label2, dets2 = batch
    # print("sample1.shape", sample1.shape)
    # print("label1.shape", label1.shape)
    # print("sample2.shape", sample2.shape)
    # print("label2.shape", label2.shape)

    ground_truth = (label1 == label2).float()
    ground_truth = torch.unsqueeze(ground_truth, 1)

    for j in np.arange(len(sample1)):
        img1 = exp_utils.torch_to_grayscale_np_img(sample1[j]).astype(np.float32)
        img2 = exp_utils.torch_to_grayscale_np_img(sample2[j]).astype(np.float32)

        affine_score, score_ratio, mconf, median_affine_error, len_matches, vis_out = object_reid_superglue.compare(img1, img2, gt=ground_truth[j], visualise=visualise, debug=debug)



        result_bin = False
        if len_matches >= min_matches:
            result_bin = True

        batch_prediction.append(result_bin)
        correct_prediction = bool(result_bin == ground_truth[j])
        if debug:
            if correct_prediction:
                print("[green]Correct prediction")
            else:
                print("[red]wrong prediction")

        if visualise:
            sns.histplot(mconf)
            plt.show()

        if visualise and vis_out is not None:            
            display(Image.fromarray(vis_out))

        results.append((affine_score, score_ratio, median_affine_error, result_bin, ground_truth[j], correct_prediction))
        
    if debug:
        if i > 10:
            break    

/media/ssd2/sruiz/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:4227: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [6]:
results = np.array(results)
print("results", results[:, -1])
print("len(results[:, -1])", results[:, -1])
acc = np.mean(results[:, -1])
print("acc", acc)

results [True True True ... True True False]

len(results[:, -1]) [True True True ... True True False]

acc 0.8445092322643343

## Results

### experiment 1:
match_threshold = 0.7
decision: matches >= 10

acc = 0.84 on seen_train

